<a href="https://colab.research.google.com/github/heena2248/Text_Mining/blob/main/nlp_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import warnings
warnings.filterwarnings('ignore')

In [ ]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df = temp_df.iloc[:100]

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
negative,58
positive,42


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
# Basic Preprocessing
# Remove tags
# lowercase
# remove stopwords

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
sw_list = stopwords.words('english')
sw_list.remove('not')
wnl=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import re
import string
def preprocess(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    punc=string.punctuation
    cleaned_text=cleaned_text.translate(str.maketrans('', '', punc)).lower()
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in sw_list])
    cleaned_text = ' '.join([wnl.lemmatize(word) for word in cleaned_text.split()])
    return cleaned_text


In [ ]:
df['review'] = df['review'].apply(preprocess)

In [ ]:
df

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
95,daniel daylewis versatile actor alive english ...,positive
96,guess would originally going least two part th...,negative
97,well like watch bad horror bmovies cause think...,negative
98,worst movie ever seen well worst probably ever...,negative


In [ ]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [ ]:
X

,review
0,one reviewer mentioned watching 1 oz episode y...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically there family little boy jake think t...
4,petter matteis love time money visually stunni...
...,...
95,daniel daylewis versatile actor alive english ...
96,guess would originally going least two part th...
97,well like watch bad horror bmovies cause think...
98,worst movie ever seen well worst probably ever...


In [ ]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
95,positive
96,negative
97,negative
98,negative


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [ ]:
y

array([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

(80, 1)

In [ ]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [ ]:
X_train_bow.shape

(80, 3851)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.75

In [ ]:
confusion_matrix(y_test,y_pred)

array([[10,  1],
       [ 4,  5]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.7

In [ ]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.65

In [ ]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.75

## Using TfIdf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.65

In [ ]:
import gensim

In [ ]:
from gensim.models import Word2Vec,KeyedVectors

In [ ]:

review = df.review.apply(gensim.utils.simple_preprocess)
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['review'], vector_size=100, window=5, min_count=2, workers=4)

In [ ]:
def get_review_vector(words, model):
    # Filter out words not in the vocabulary
    valid_words = [word for word in words if word in model.wv.key_to_index]
    if not valid_words:
        return np.zeros(model.vector_size)  # Return a zero vector if no valid words are found
    return np.mean([model.wv[word] for word in valid_words], axis=0)

# Apply the function to each review
df['review_vector'] = df['review'].apply(lambda x: get_review_vector(x, word2vec_model))

In [ ]:

X = np.array(df['review_vector'].tolist())

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.35


In [ ]:
# Finding similar words and similarity/polarity score between words
model.wv.most_similar("product")

[('message', 0.410366415977478),
 ('mean', 0.36864906549453735),
 ('later', 0.33612048625946045),
 ('free', 0.3336467742919922),
 ('cristy', 0.3192509710788727),
 ('gave', 0.3187011480331421),
 ('impressive', 0.31825244426727295),
 ('soundtrack', 0.31751886010169983),
 ('aware', 0.3168780505657196),
 ('hot', 0.3148908317089081)]

In [ ]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

In [ ]:
# Remove stopwords

X_train = X_train['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))
# Remove stopwords

X_test = X_test['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
import spacy
import en_core_web_sm
# Load the spacy model. This takes a few seconds.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc = nlp(X_train.values[0])
print(doc.vector)

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
input_arr = []
for item in X_train.values:
    doc = nlp(item)
    input_arr.append(doc.vector)

In [ ]:
input_arr = np.array(input_arr)

In [ ]:
input_arr.shape

In [ ]:
input_test_arr = []
for item in X_test.values:
    doc = nlp(item)
    input_test_arr.append(doc.vector)

In [ ]:
input_test_arr = np.array(input_test_arr)

In [ ]:
input_test_arr.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
gnb.fit(input_arr,y_train)

In [ ]:
y_pred = gnb.predict(input_test_arr)
accuracy_score(y_test,y_pred)